In [14]:
import os
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [2]:
from graph_bridges.models.backward_rates.backward_rate import GaussianTargetRateImageX0PredEMA
from graph_bridges.data.dataloaders import DoucetTargetData
from graph_bridges.models.samplers.sampling import ReferenceProcess
from graph_bridges.models.reference_process.ctdd_reference import ReferenceProcess
from graph_bridges.models.losses.ctdd_losses import GenericAux
from graph_bridges.models.schedulers.scheduling_ctdd import CTDDScheduler

from graph_bridges.models.pipelines.pipelines_utils import create_pipelines
from graph_bridges.models.schedulers.scheduling_utils import create_scheduler
from graph_bridges.models.backward_rates.backward_rate_utils import create_model
from graph_bridges.models.reference_process.reference_process_utils import create_reference
from graph_bridges.data.dataloaders_utils import create_dataloader
from graph_bridges.models.losses.loss_utils import create_loss

from graph_bridges.configs.graphs.lobster.config_base import BridgeConfig

In [3]:
config = BridgeConfig()
device = torch.device(config.device)

In [4]:
#=================================================================
# CREATE OBJECTS FROM CONFIGURATION

data_dataloader: DoucetTargetData
model : GaussianTargetRateImageX0PredEMA
reference_process: ReferenceProcess
loss : GenericAux
scheduler:CTDDScheduler

data_dataloader = create_dataloader(config,device)
model = create_model(config,device)
reference_process = create_reference(config,device)
loss = create_loss(config,device)
scheduler = create_scheduler(config,device)

Scheduler


In [5]:
#=================================================================
sample_ = data_dataloader.sample(config.number_of_paths, device)
minibatch = sample_.unsqueeze(1).unsqueeze(1)

In [16]:
# TIME ===========================================================
#ts = torch.rand((minibatch.shape[0],), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
B = minibatch.shape[0]
ts = torch.rand((B,), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
#==========

x_t, x_tilde, qt0, rate = scheduler.add_noise(minibatch,reference_process,ts,device,return_dict=False)
scheduler_noise_output = scheduler.add_noise(minibatch,reference_process,ts,device,return_dict=True)

x_logits,p0t_reg,p0t_sig,reg_x = model.forward(minibatch,ts,x_tilde)
model_forward_output = model.forward(minibatch,ts,x_tilde,return_dict=True)

loss_ = loss.calc_loss(minibatch,x_tilde,qt0,rate,x_logits,reg_x,p0t_sig,p0t_reg,device)

In [9]:
config.initialize()

In [12]:
#config.best_model_path
#config.sinkhorn_plot_path

'C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\Codes\\graph-bridges\\results\\graph\\lobster\\testing\\sinkhorn_{0}.tr'

In [17]:
from diffusers.optimization import get_cosine_schedule_with_warmup
num_batches = 20

optimizer = torch.optim.AdamW(model.parameters(), lr=config.optimizer.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.optimizer.lr_warmup_steps,
    num_training_steps=(num_batches * config.optimizer.num_epochs),
)

In [18]:
loss_ = loss.calc_loss(minibatch,x_tilde,qt0,rate,x_logits,reg_x,p0t_sig,p0t_reg,device)

In [22]:
global_step = 0

# Now you train the model
for epoch in range(config.optimizer.num_epochs):
    progress_bar = tqdm(total=num_batches)
    progress_bar.set_description(f"Epoch {epoch}")

    for step in range(num_batches):
        # Obtain data
        sample_ = data_dataloader.sample(config.number_of_paths, device)
        minibatch = sample_.unsqueeze(1).unsqueeze(1)
        B = minibatch.shape[0]
        
        # Sample a random timestep for each image
        ts = torch.rand((B,), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
        
        # Sample noise to add to the images
        # Add noise to the clean images according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        
        #noisy_images = scheduler.add_noise(clean_images, noise, timesteps)

        x_t, x_tilde, qt0, rate = scheduler.add_noise(minibatch,
                                                      reference_process,
                                                      ts,
                                                      device,
                                                      return_dict=False)
        
        #scheduler_noise_output = scheduler.add_noise(minibatch,
        #                                             reference_process,
        #                                             ts,
        #                                             device,
        #                                             return_dict=True)
        # Predict the noise residual
        #noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
        #loss = F.mse_loss(noise_pred, noise)
        #accelerator.backward(loss)
            
        x_logits,p0t_reg,p0t_sig,reg_x = model.forward(minibatch,
                                                       ts,
                                                       x_tilde)
            
        #model_forward_output = model.forward(minibatch,
        #                                     ts,
        #                                     x_tilde,
        #                                     return_dict=True)
            
        loss_ = loss.calc_loss(minibatch,
                               x_tilde,
                               qt0,
                               rate,
                               x_logits,
                               reg_x,
                               p0t_sig,
                               p0t_reg,
                               device)
        
        loss_.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        logs = {"loss": loss_.detach().item(), 
                "lr": lr_scheduler.get_last_lr()[0], 
                "step": global_step}
        
        progress_bar.set_postfix(**logs)
        #accelerator.log(logs, step=global_step)
        global_step += 1


Epoch 0:   5%|███▊                                                                        | 1/20 [00:09<03:05,  9.76s/it]

Epoch 1: 100%|█████████████████████████████████████████████| 20/20 [00:00<00:00, 40.90it/s, loss=103, lr=8.2e-6, step=39]

Epoch 3: 100%|████████████████████████████████████████████| 20/20 [00:00<00:00, 38.87it/s, loss=113, lr=1.62e-5, step=79]

Epoch 4:  30%|█████████████▌                               | 6/20 [00:00<00:00, 36.19it/s, loss=114, lr=1.74e-5, step=85]


Epoch 5: 100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 41.63it/s, loss=102, lr=2.42e-5, step=119]

Epoch 7: 100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 41.82it/s, loss=108, lr=3.22e-5, step=159]

Epoch 8:  65%|███████████████████████████▎              | 13/20 [00:00<00:00, 35.45it/s, loss=99.7, lr=3.48e-5, step=172]


Epoch 9: 100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 40.53it/s, loss=109, lr=4.02e-5, step=199]

Epoch 11: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 40.73it/s, loss=97.4, lr=4.82e-5, step=239]

Epoch 12: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 36.47it/s, loss=94.6, lr=5.22e-5, step=259]


Epoch 13: 100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 40.77it/s, loss=111, lr=5.62e-5, step=279]

Epoch 15: 100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 39.82it/s, loss=113, lr=6.42e-5, step=319]

Epoch 17: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 41.89it/s, loss=94.5, lr=7.22e-5, step=359]

Epoch 18:  30%|████████████▌                             | 6/20 [00:00<00:00, 38.81it/s, loss=92.5, lr=7.34e-5, step=365]


Epoch 19: 100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 39.56it/s, loss=112, lr=8.02e-5, step=399]

Epoch 21: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 38.49it/s, loss=86.1, lr=8.82e-5, step=439]

Epoch 22:  70%|██████████████████████████████             | 14/20 [00:00<00:00, 35.11it/s, loss=104, lr=9.1e-5, step=453]


...

Epoch 29: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 42.31it/s, loss=79.4, lr=9.03e-5, step=599]

Epoch 31: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 39.03it/s, loss=92.1, lr=8.16e-5, step=639]

Epoch 32:  40%|████████████████▊                         | 8/20 [00:00<00:00, 32.52it/s, loss=79.5, lr=7.96e-5, step=647]


Epoch 33: 100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 41.16it/s, loss=86.5, lr=7.1e-5, step=679]

Epoch 35: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 39.87it/s, loss=83.5, lr=5.91e-5, step=719]

Epoch 36:  80%|████████████████████████████████▊        | 16/20 [00:00<00:00, 36.47it/s, loss=91.6, lr=5.41e-5, step=735]


Epoch 37: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 39.02it/s, loss=83.1, lr=4.65e-5, step=759]

Epoch 39: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 40.34it/s, loss=93.5, lr=3.43e-5, step=799]

Epoch 41: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 39.85it/s, loss=87.5, lr=2.29e-5, step=839]

Epoch 42:  10%|████▏                                     | 2/20 [00:00<00:00, 33.11it/s, loss=79.2, lr=2.24e-5, step=841]


Epoch 43: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 39.38it/s, loss=86.2, lr=1.33e-5, step=879]

Epoch 45: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 40.14it/s, loss=80.5, lr=6.03e-6, step=919]

Epoch 46:  50%|█████████████████████                     | 10/20 [00:00<00:00, 34.92it/s, loss=114, lr=4.63e-6, step=929]


Epoch 47: 100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 40.20it/s, loss=99.9, lr=1.49e-6, step=959]

Epoch 49: 100%|████████████████████████████████████████| 20/20 [00:14<00:00, 39.92it/s, loss=95.8, lr=9.87e-10, step=999]